<a href="https://colab.research.google.com/github/almir-martins/migrando-pacotes-excel/blob/main/Simulando_Pacotes_Correios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulação descontos e contrapartida na troca de pacotes de serviços

### Simulação de migração dos pacotes de serviços antigos para os  pacotes da Nova Política Comercial usando a planilha fornecida em Excel. 

### Usando a biblioteca <a href="https://github.com/burnash/gspread"><code>gspread</code></a> de código aberto para interagir com o Planilhas Google.



In [ ]:
!pip install --upgrade gspread

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


### Importar a biblioteca
### Autenticação no Google Planilhas.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

# Pega os cabeçalhos das colunas da planilha SEDEX na versão do cliente

In [ ]:
def pega_cabecalhos_sedex(arquivo):    
  # Seleciona a aba
  aba = 'Almir simulação'

  # Seleciona a planilha
  planilha = arquivo.worksheet(aba)

  # Pega os cabeçalhos
  cabecalhos = planilha.get('A3:M3')

  # Converte para dataframe
  df_cabecalhos = pd.DataFrame.from_records(cabecalhos)

  return list(df_cabecalhos.loc[0])

# Método para formatar tabela de SEDEX

In [ ]:
def formata_tabela_sedex(nome_planilha, arquivo): 
  # Seleciona Planilha do pacote do cliente
  planilha = arquivo.worksheet(nome_planilha)

  # Tabela de preços do cliente (PAC)
  tabela_sedex = planilha.get('A5:U18')  

  # Converte para dataframe
  df_tabela_sedex = pd.DataFrame.from_records(tabela_sedex)

  # Apaga a primeira linha
  df_tabela_sedex = df_tabela_sedex.drop([0,0])

  # Apaga as colunas desnecessárias
  df_tabela_sedex.drop(df_tabela_sedex.columns[[2,3,4,6,7,8,14,20]], axis=1, inplace=True)

  # Pega os cabeçalhos definitivos
  df_tabela_sedex.columns = pega_cabecalhos_sedex(arquivo)

  return df_tabela_sedex


# Pega os cabeçalhos das colunas da planilha PAC na versão do cliente

In [ ]:
def pega_cabecalhos_pac(arquivo):    
  # Seleciona a aba
  aba = 'Almir simulação'

  # Seleciona a planilha
  planilha = arquivo.worksheet(aba)

  # Pega os cabeçalhos
  cabecalhos = planilha.get('A21:L21')

  # Converte para dataframe
  df_cabecalhos = pd.DataFrame.from_records(cabecalhos)

  return list(df_cabecalhos.loc[0])

# Método para formatar tabela de PAC

In [ ]:
def formata_tabela_pac(nome_planilha, arquivo): 
  # Seleciona Planilha do pacote do cliente
  planilha = arquivo.worksheet(nome_planilha)

  # Tabela de preços do cliente (PAC)
  tabela_pac = planilha.get('A24:Q36')  

  # Converte para dataframe
  df_tabela_pac = pd.DataFrame.from_records(tabela_pac)

  # Apaga a primeira linha
  df_tabela_pac = df_tabela_pac.drop([0,0])

  # Apaga as colunas desnecessárias
  df_tabela_pac.drop(df_tabela_pac.columns[[2,3,4,10,16]], axis=1, inplace=True)

  # Pega os cabeçalhos definitivos
  df_tabela_pac.columns = pega_cabecalhos_pac(arquivo)

  return df_tabela_pac


# Fábrica de pacotes

## Constrói as tabelas de SEDEX e PAC do pacote requisitado

In [ ]:
def gera_pacote(pacote):
  # Abre o arquivo
  arquivo = gc.open('Simulador de migração - Pacotes Python')

  # constrói a tabela SEDEX
  tabela_SEDEX = formata_tabela_sedex(pacote, arquivo)

  # constrói a tabela PAC
  tabela_PAC = formata_tabela_pac(pacote, arquivo)  

  return tabela_SEDEX, tabela_PAC

In [ ]:
# Pacote de origem
pacote_cliente = 'ENC_2.1'

pac_cliente, sedex_cliente = gera_pacote(pacote_cliente)

pacote_1 = 'BRONZE1'
pacote_2 = 'PRATA1'



print(pac_cliente.to_string(index=False))
print(sedex_cliente.to_string(index=False))